In [15]:
import pandas as pd
import numpy as np
import sys
from common import commons
home = commons.home
from features_preprocess import get_winid
from common.commons import rename_features
import os
import re

In [16]:
def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.ix[merged['before_tss'].isnull()==False, 'before_tss'] = merged.ix[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.ix[before_ix,'tss'] = merged.ix[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.ix[after_ix,'tss'] = merged.ix[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss']) 
    merged.drop(['before_tss','after_tss','tss','dist_to_before_tss','dist_to_after_tss'],axis=1,inplace=True)
    merged.dropna(axis=0,subset=['id'],inplace=True)
    return merged


In [25]:
dataset = commons.dataset
if dataset == 'AD_CpG':
    type_name = commons.type_name  ## amyloid, cerad, tangles
    with_cell_type = commons.with_cell_type ## with or without
    dataset = dataset+'/'+type_name+with_cell_type
with pd.HDFStore(home+'data/'+dataset+'/all_sites_winid','r') as h5s:
    all_sites = h5s['all_sites_winid']
all_sites.reset_index(drop=True,inplace=True)

In [27]:
feature_dir = home+'data/features/'+dataset+'/'
files = os.listdir(feature_dir)
pattern = '.*all.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]

In [28]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    all_sites = pd.concat([all_sites,feature],axis=1)

303
31
267
317
735
73
80


In [29]:
rename_features(all_sites)

In [30]:
all_sites.drop(['start','end'],axis=1,inplace=True)

In [32]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS','GWAVA']
#merge with additional features
with pd.HDFStore(feature_dir+'addtional_features','r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        all_sites = pd.concat([all_sites,feature_frame],axis=1)
all_sites = all_sites.loc[:,~all_sites.columns.duplicated()]
all_sites['chr'] = all_sites['chr'].astype('i8')

In [33]:
#nearest tss distance    
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'data/commons/tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
all_sites = nearest_tss(tss,all_sites)


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


/home/ec2-user/volume/git/EnsembleCpG/code/features_preprocess/get_winid.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].apply(lambda x: int(x[3:]))
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/i

In [46]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label,winid,A549-BHLHE40,A549-CEBPB,...,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,GWAVA_region_score,GWAVA_tss_score,GWAVA_unmatched_score,GWAVA_dist_to_nearest_snp,dist_to_nearest_tss
0,cg25605174,1,1106175.0,0.179798,0.671729,0.828180,0.0,5531.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.22,0.410,0.40,63.0,1719.0
1,cg12286462,1,1358109.0,0.260215,0.610197,0.794203,0.0,6791.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.45,0.390,0.70,0.0,1285.0
2,cg24470133,1,1483206.0,0.002705,0.958542,0.960068,0.0,7417.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.28,0.060,0.16,19.0,7466.0
3,cg22373622,1,1489544.0,16.545792,0.000055,0.918745,1.0,7448.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.25,0.180,0.18,6.0,13804.0
4,cg00007800,1,1838518.0,0.280518,0.596597,0.794176,0.0,9193.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.32,0.400,0.20,2.0,7747.0
5,cg06801943,1,2101479.0,0.538160,0.463540,0.645085,0.0,10508.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.21,0.280,0.50,0.0,1193.0
6,cg18934822,1,2191402.0,15.202460,0.000110,0.378193,1.0,10958.0,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.45,0.070,0.06,5.0,31269.0
7,cg19694770,1,2407304.0,0.668186,0.414073,0.923682,0.0,12037.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.33,0.370,0.52,40.0,449.0
8,cg18123385,1,2938122.0,0.033714,0.854392,0.575633,0.0,14691.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.53,0.400,0.76,21.0,77.0
9,cg14008571,1,3038564.0,0.368172,0.544277,0.702007,0.0,15193.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49,0.520,0.43,34.0,5974.0


In [15]:
with pd.HDFStore(home+'data/'+dataset+'/all_features','w') as h5s:
    h5s['all_features'] = all_sites

In [9]:
###all 450K sites features, only need to RUN ONCE
dataset = 'Cd'
feature_dir = home+'data/features/'+dataset+'/'
all_450_features = home+'data/'+dataset+'/all_450k_features'
if dataset == 'AD_CpG':
    type_name = commons.type_name  ## amyloid, cerad, tangles
    with_cell_type = commons.with_cell_type ## with or without
    dataset = dataset+'/'+type_name+with_cell_type


In [10]:
with pd.HDFStore(home+'data/'+dataset+'/all_450k_sites_winid','r') as h5s:
    all_sites = h5s['all_450k_sites_winid']
all_sites.reset_index(drop=True,inplace=True)

In [11]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,start,winid,end
0,cg13869341,1,15865,0.0426,0.297000,0.862405,15801,80,16000
1,cg24669183,1,534242,-0.0561,0.421300,0.803156,534201,2672,534400
2,cg15560884,1,710097,-0.0058,0.788800,0.662852,710001,3551,710200
3,cg01014490,1,714177,-0.0243,0.375500,0.016537,714001,3571,714200
4,cg17505339,1,720865,-0.0853,0.027480,0.967550,720801,3605,721000
5,cg05898754,1,805102,0.0182,0.659200,0.177819,805001,4026,805200
6,cg03128332,1,805338,0.0092,0.882000,0.045307,805201,4027,805400
7,cg16619049,1,805541,0.0007,0.991300,0.283757,805401,4028,805600
8,cg05475702,1,812248,0.0401,0.038670,0.472167,812201,4062,812400
9,cg18147296,1,812539,-0.0007,0.976800,0.311583,812401,4063,812600


In [12]:
#feature_dir = home+'data/features/'+dataset.split('/')[0]+'/'
files = os.listdir(feature_dir)
pattern = '.*all_450k.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]

In [13]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    all_sites = pd.concat([all_sites,feature],axis=1)

317
31
735
303
73
267
80


In [14]:
rename_features(all_sites)

In [15]:
all_sites.drop(['start','end'],axis=1,inplace=True)

In [16]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS','GWAVA']
#merge with additional features
with pd.HDFStore(feature_dir+'all_450k_addtional_features','r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        all_sites = pd.concat([all_sites,feature_frame],axis=1)
all_sites = all_sites.loc[:,~all_sites.columns.duplicated()]
all_sites['chr'] = all_sites['chr'].astype('i8')

In [18]:
#nearest tss distance 
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'data/commons/tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
all_sites = nearest_tss(tss,all_sites)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [20]:
with pd.HDFStore(all_450_features,'w') as h5s:
    h5s['all_450k_features'] = all_sites.drop(['pvalue','beta'],axis=1)

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['id', 'dist_to_nearest_tss']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
import pandas as pd
with pd.HDFStore('/home/ec2-user/volume/git/EnsembleCpG/data/RNASeq/ENCFF939IHN','r') as h5s:
    x = h5s['ENCFF939IHN']

In [14]:
x

,winid,ENCFF939IHN_RNASeq_counts
